In [1]:
import logging
import sys

from src.simulator.factory import Factory


logging.basicConfig(
     filename='run.log',
     filemode='w',
     level=logging.INFO,
     format='%(asctime)s - %(name)s - %(levelname)-7s - %(message)s',
     datefmt='%H:%M:%S'
 )
logger = logging.getLogger(__name__)


logger.info('Starting simulation')
factory = Factory.from_config('config/factory.yml')
factory.run(2)
factory.plot()

In [ ]:
# TODO: X Filling process
# TODO: failure process
# TODO: Sensor based alarms/events.
# TODO: Adjust all the values according to given wishes
# TODO: Locking mechanism when putting stuff
# TODO: Add/improve monitoring
# TODO: Why part counter/product not in plots?
# TODO: Toggle monitor data collection

Missing:
- Planned maintenance X
- Material / batch ids (materiaali-id voi olla string) X
- Collectng events (debug / info / warning / error) --> logs not for now, separate variable when necessary, error/stop code most critical for now (integer) --> exact variables in Excel
- Mappings from string states into integers / boolean X
- Randomization of fails - "global fail proba based on last maintenance"
- Integration: Pull/push? Taking snapshots of the machine state vs. pushing e.g. error X
- Cumulative: Consumable X
- Program: add part counter X / target / postprocessing step that identifies failed  
- Randomization of time
- Running in AWS / Noux environments / CSV

2023 csv:t, jatketaan kumulatiivisia


In [ ]:
# TODO: Viesti, jossa laitettu mukaan factory.yml ja kerrottu, minkälaisia asioita pitää täyttää
# TODO: 